## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,41.72,88,74,13.44,broken clouds
1,1,Kodiak,US,57.7900,-152.4072,51.31,93,90,18.41,light rain
2,2,Chuy,UY,-33.6971,-53.4616,53.65,92,73,12.84,broken clouds
3,3,Butaritari,KI,3.0707,172.7902,83.61,72,9,14.56,light rain
4,4,Abu Zabad,SD,12.3500,29.2500,73.15,81,86,9.55,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 71


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
47,47,Arraial Do Cabo,BR,-22.9661,-42.0278,70.72,87,41,30.11,scattered clouds
77,77,Ilhabela,BR,-23.7781,-45.3581,70.61,93,82,3.83,broken clouds
96,96,Cambui,BR,-22.6122,-46.0575,70.54,47,85,2.35,overcast clouds
222,222,Nouadhibou,MR,20.9310,-17.0347,70.70,89,19,18.54,few clouds
228,228,Sikonge,TZ,-5.6333,32.7667,70.18,50,48,9.40,scattered clouds
284,284,Natal,BR,-5.7950,-35.2094,70.45,94,20,3.44,few clouds
350,350,Camacha,PT,33.0833,-16.3333,70.56,83,20,5.75,few clouds
364,364,Dondo,MZ,-19.6094,34.7431,70.97,83,14,3.27,few clouds
377,377,Lagoa,PT,39.0500,-27.9833,70.95,86,66,11.92,broken clouds
403,403,La Joya,MX,18.6667,-99.4667,70.39,96,100,4.79,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                20
City                   20
Country                20
Lat                    20
Lng                    20
Max Temp               20
Humidity               20
Cloudiness             20
Wind Speed             20
Current Description    20
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                20
City                   20
Country                20
Lat                    20
Lng                    20
Max Temp               20
Humidity               20
Cloudiness             20
Wind Speed             20
Current Description    20
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
47,Arraial Do Cabo,BR,70.72,scattered clouds,-22.9661,-42.0278,
77,Ilhabela,BR,70.61,broken clouds,-23.7781,-45.3581,
96,Cambui,BR,70.54,overcast clouds,-22.6122,-46.0575,
222,Nouadhibou,MR,70.70,few clouds,20.9310,-17.0347,
228,Sikonge,TZ,70.18,scattered clouds,-5.6333,32.7667,
284,Natal,BR,70.45,few clouds,-5.7950,-35.2094,
350,Camacha,PT,70.56,few clouds,33.0833,-16.3333,
364,Dondo,MZ,70.97,few clouds,-19.6094,34.7431,
377,Lagoa,PT,70.95,broken clouds,39.0500,-27.9833,
403,La Joya,MX,70.39,overcast clouds,18.6667,-99.4667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))